In [6]:
import time
from torch.utils.data import DataLoader
import torch
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from mask_generator.dataset import ImageMaskDataset
from mask_generator.transform_manager import TransformManager
from mask_generator.utils.dataset_utils import DatasetLoaderFactory

In [7]:
DATASET_DIR = "../Datasets/train/simu_v0"

pairs_path = DatasetLoaderFactory.get_loader(
        DATASET_DIR,
    ).get_pairs()

print(f"Successfully loaded {len(pairs_path)} pairs of images and masks.")

Successfully loaded 4817 pairs of images and masks.


In [10]:
augmentations_list = ["geometry", "dropout", "color_invariance", "color_variation", "blur", "noise", "weather"]

filters = [32, 64, 128, 256]

transform_manager = TransformManager(seed=42, num_encoders=len(filters))
train_ds = ImageMaskDataset(

    pairs_path=pairs_path,
    transform=transform_manager.get_eval_transform(),
)

batch_size = 16

workers_to_test = [4, 8, 16, 24]

results = []

for num_workers in workers_to_test:
    loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
        persistent_workers=True,
        prefetch_factor=4,
        shuffle=True,
    )

    # Warm-up (pour initialiser workers, éviter le biais temps 1er batch)
    for _ in loader:
        break

    start_time = time.time()
    for i, (x, y, _, _) in enumerate(loader):
        if i == 50:  # 50 batches testés, tu peux changer
            break
    elapsed = time.time() - start_time

    avg_batch_time = elapsed / 50
    print(f"num_workers={num_workers} | avg batch load time: {avg_batch_time:.4f} s")
    results.append((num_workers, avg_batch_time))

# Affiche résumé
print("\nRésultats complets :")
for w, t in results:
    print(f"{w} workers -> {t:.4f} s/batch")

num_workers=4 | avg batch load time: 0.0458 s
num_workers=8 | avg batch load time: 0.0424 s
num_workers=16 | avg batch load time: 0.0652 s
num_workers=24 | avg batch load time: 0.0889 s

Résultats complets :
4 workers -> 0.0458 s/batch
8 workers -> 0.0424 s/batch
16 workers -> 0.0652 s/batch
24 workers -> 0.0889 s/batch


In [ ]:
# Without Transforms
# num_workers=4 | avg batch load time: 0.0343 s
# num_workers=8 | avg batch load time: 0.0455 s
# num_workers=16 | avg batch load time: 0.0680 s
# num_workers=24 | avg batch load time: 0.0806 s

# Résultats complets :
# 4 workers -> 0.0343 s/batch
# 8 workers -> 0.0455 s/batch
# 16 workers -> 0.0680 s/batch
# 24 workers -> 0.0806 s/batch

# With Eval Transforms
# num_workers=4 | avg batch load time: 0.0458 s
# num_workers=8 | avg batch load time: 0.0424 s
# num_workers=16 | avg batch load time: 0.0652 s
# num_workers=24 | avg batch load time: 0.0889 s

# Résultats complets :
# 4 workers -> 0.0458 s/batch
# 8 workers -> 0.0424 s/batch
# 16 workers -> 0.0652 s/batch
# 24 workers -> 0.0889 s/batch

# With train transform
# num_workers=4 | avg batch load time: 0.2736 s
# num_workers=8 | avg batch load time: 0.1826 s
# num_workers=16 | avg batch load time: 0.3284 s
# num_workers=24 | avg batch load time: 0.4190 s

# Résultats complets :
# 4 workers -> 0.2736 s/batch
# 8 workers -> 0.1826 s/batch
# 16 workers -> 0.3284 s/batch
# 24 workers -> 0.4190 s/batch

In [28]:
from mask_generator.augmentation_factory import AugmentationFactory
import numpy as np

factory = AugmentationFactory(seed=42)
for name, transform_obj in factory.transformations.items():
    start = time.time()
    # Crée un transform unique
    transform = transform_obj()
    # Simule une application sur une image factice
    dummy = {"image": np.random.randint(0, 256, (256, 256, 3), dtype=np.uint8)}
    for _ in range(100):
        _ = transform(**dummy)
    elapsed = time.time() - start
    print(f"Transform {name}: avg time per call {(elapsed/100):.4f}s")

Transform blur: avg time per call 0.0001s
Transform noise: avg time per call 0.0005s
Transform dropout: avg time per call 0.0001s
Transform geometry: avg time per call 0.0002s
Transform color_invariance: avg time per call 0.0000s
Transform color_variation: avg time per call 0.0001s
Transform weather: avg time per call 0.0011s


In [ ]:
import torch.profiler

with torch.profiler.profile(
    activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
    record_shapes=True,
    profile_memory=True,
    with_stack=True
) as prof:
    print(prof.key_averages().table(sort_by="cuda_time_total"))